<a href="https://colab.research.google.com/github/hellojohnkim/mmai823/blob/main/sandbox_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import yfinance as yf
import pandas as pd
import warnings
import os
warnings.filterwarnings("ignore")

In [2]:
## S&P500 constituents list

## scraping wikipedia table to get list of tickers of companies aka constituents

# There are 2 tables on the Wikipedia page, we want the first table
# second table shows the evolution of the changes

components = \
pd.read_html('https://en.wikipedia.org/wiki/List_of_S&P_500_companies')

first_table = components[0]
second_table = components[1]
stock = first_table
stock.Symbol
stock
sp500_tickers = list(stock['Symbol'])

In [3]:
stock

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
499,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
500,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
501,ZION,Zions Bancorporation,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


In [4]:
# extract stock description function
def get_stock_descriptions(sp500_tickers):
    descriptions = []

    for ticker in sp500_tickers:
        try:
            # create Ticker
            ticker = yf.Ticker(ticker)

            # fetch stock info
            stock_info = ticker.info

            # fetch 'longBusinessSummary'
            description = stock_info.get('longBusinessSummary', None)

            # add to list
            descriptions.append(description)
        except Exception as e:
            print(f"Error fetching data for {ticker}: {str(e)}")
            descriptions.append(None)

    data = pd.DataFrame({'Symbol': sp500_tickers, 'description': descriptions})

    return data

In [5]:
#create dataframe
stock_description = get_stock_descriptions(sp500_tickers)

stock_description

,Symbol,description
0,MMM,3M Company provides diversified technology ser...
1,AOS,A. O. Smith Corporation manufactures and marke...
2,ABT,"Abbott Laboratories, together with its subsidi..."
3,ABBV,"AbbVie Inc. discovers, develops, manufactures,..."
4,ACN,"Accenture plc, a professional services company..."
...,...,...
498,YUM,"Yum! Brands, Inc., together with its subsidiar..."
499,ZBRA,"Zebra Technologies Corporation, together with ..."
500,ZBH,"Zimmer Biomet Holdings, Inc., together with it..."
501,ZION,"Zions Bancorporation, National Association pro..."


In [13]:
#add security name
company_name = stock[['Symbol', 'Security']]
company_name
stock_description = pd.merge(stock_description, company_name, how='inner', on='Symbol')
stock_description.head()

,Symbol,description,Security
0,MMM,3M Company provides diversified technology ser...,3M
1,AOS,A. O. Smith Corporation manufactures and marke...,A. O. Smith
2,ABT,"Abbott Laboratories, together with its subsidi...",Abbott
3,ABBV,"AbbVie Inc. discovers, develops, manufactures,...",AbbVie
4,ACN,"Accenture plc, a professional services company...",Accenture


In [18]:
stock_description.isnull().sum()

Symbol         0
description    2
Security       0
dtype: int64

In [19]:
stock_description = stock_description.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [20]:
stock_description.dropna(subset=['description'], inplace=True)
stock_description.reset_index(drop=True, inplace=True)
stock_description.drop_duplicates(subset=['description'], inplace=True)

In [21]:
stock_description.head()

,Symbol,description,Security
0,MMM,3M Company provides diversified technology ser...,3M
1,AOS,A. O. Smith Corporation manufactures and marke...,A. O. Smith
2,ABT,"Abbott Laboratories, together with its subsidi...",Abbott
3,ABBV,"AbbVie Inc. discovers, develops, manufactures,...",AbbVie
4,ACN,"Accenture plc, a professional services company...",Accenture


In [22]:
stock_description.shape

(498, 3)

In [6]:
# add more stock information
def add_stock_info(df):
    info_columns_mapper = {

        # Information related
        'sector': 'sector',
        'industry': 'industry',
        'longBusinessSummary': 'description',

        # Trading information related
        'sharesOutstanding': 'sharesOutstanding',
        'averageVolume10days': 'averageVolume10days',
        'averageVolume': 'averageVolume',
        'heldPercentInstitutions': 'heldPercentInstitutions',
        'shortRatio': 'shortRatio',
        'sharesPercentSharesOut': 'sharesPercentSharesOut',
        'shortPercentOfFloat': 'shortPercentOfFloat',

        # Price related
        'marketCap': 'marketCap',
        'currentPrice': 'currentPrice',
        'fiftyDayAverage': 'fiftyDayAverage',
        'twoHundredDayAverage': 'twoHundredDayAverage',
        'fiftyTwoWeekHigh': 'fiftyTwoWeekHigh',
        'fiftyTwoWeekLow': 'fiftyTwoWeekLow',
        'SandP52WeekChange': 'SandP52WeekChange',
        '52WeekChange': '52WeekChange',
        'ytdReturn': 'ytdReturn',
        'fiveYearAverageReturn': 'fiveYearAverageReturn',
        'beta': 'beta',

        # Cash creation, sales related (ttm)
        'totalRevenue': 'totalRevenue',
        'grossProfits': 'grossProfits',
        'revenuePerShare': 'revenuePerShare',
        'ebitda': 'EBITDA',
        'ebitdaMargins': 'ebitdaMargins',

        # Financial status related (mrq)
        'debtToEquity': 'debtToEquity',
        'operatingCashflow': 'operatingCashflow',
        'freeCashflow': 'freeCashflow',
        'totalCashPerShare': 'totalCashPerShare',
        'currentRatio': 'currentRatio',
        'quickRatio': 'quickRatio',
        'overallRisk': 'overallRisk',

        # Management efficiency related
        'returnOnAssets': 'returnOnAssets',
        'returnOnEquity': 'returnOnEquity',
        'grossMargins': 'grossMargins',
        'operatingMargins': 'operatingMargins',
        'profitMargins': 'profitMargins',

        # Corporate assets related
        'totalCash': 'totalCash',
        'totalDebt': 'totalDebt',

        # Corporate value related
        'priceToBook': 'priceToBook',
        'enterpriseValue': 'enterpriseValue',
        'enterpriseToRevenue': 'enterpriseToRevenue',
        'enterpriseToEbitda': 'enterpriseToEbitda',
        'forwardEps': 'forwardEps',
        'trailingEps': 'trailingEps',
        'priceToSalesTrailing12Months': 'priceToSalesTrailing12Months',
        'forwardPE': 'forwardPE',
        'trailingPE': 'trailingPE',

        # Growth related
        'revenueGrowth': 'revenueGrowth',
        'earningsGrowth': 'earningsGrowth',
        'earningsQuarterlyGrowth': 'earningsQuarterlyGrowth',
        'revenueQuarterlyGrowth': 'revenueQuarterlyGrowth',
        'heldPercentInsiders': 'heldPercentInsiders',

        'Research Development': 'R&Dcost',
        'Net Income': 'NetIncome',
        'Gross Profit': 'GrossProfit',
        'Operating Income': 'OperatingIncome',
        'Total Revenue': 'TotalRevenue',
        'Cost Of Revenue': 'CostofRevenue',

        'Total Liab': 'totalLiability',
        'Total Stockholder Equity': 'totalStockholderEquity',
        'Total Assets': 'totalAssets',
    }

    for i, row in df.iterrows():
        sp500_tickers = row['Symbol']  # get symbol/ticker
        ticker = yf.Ticker(sp500_tickers)  # create Ticker object

        # fetch stock info with ticker object
        stock_info = ticker.info

        # extract data listed in column mapper into dataframe
        for key, value in info_columns_mapper.items():
            if key in stock_info:
                df.at[i, value] = stock_info[key]

    return df


In [7]:
stock_info = add_stock_info(stock)

stock_info

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded,sector,industry,...,trailingEps,priceToSalesTrailing12Months,forwardPE,revenueGrowth,earningsGrowth,earningsQuarterlyGrowth,heldPercentInsiders,SandP52WeekChange,52WeekChange,trailingPE
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902,Industrials,Conglomerates,...,-12.63,1.766793,10.252209,-0.008,0.744,0.747,0.00115,NaN,NaN,NaN
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916,Industrials,Specialty Industrial Machinery,...,3.69,3.353653,20.290993,0.056,NaN,NaN,0.00833,0.315025,0.323049,23.810297
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888,Healthcare,Medical Devices,...,3.26,4.925335,22.279842,0.015,0.550,0.543,0.00539,0.315025,0.225131,34.923313
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888),Healthcare,Drug Manufacturers - General,...,2.72,5.820265,14.669672,-0.054,-0.669,-0.668,0.00139,0.315025,0.174945,65.797790
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989,Technology,Information Technology Services,...,10.78,3.630712,27.602215,0.030,0.006,0.004,0.00148,0.315025,0.512840,34.694805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997,Consumer Cyclical,Restaurants,...,5.60,5.431304,21.016153,0.008,0.243,0.248,0.00149,NaN,NaN,24.393750
499,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969,Technology,Communication Equipment,...,5.72,3.200353,20.278408,-0.329,-0.911,-0.909,0.00641,NaN,NaN,49.916084
500,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927,Healthcare,Medical Devices,...,4.88,3.456157,14.339472,0.063,NaN,NaN,0.00123,NaN,NaN,25.534836
501,ZION,Zions Bancorporation,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873,Financial Services,Banks—Regional,...,4.35,1.976228,8.575432,-0.119,-0.575,-0.556,0.01756,NaN,NaN,9.147127


In [9]:
stock_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 58 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Symbol                        503 non-null    object 
 1   Security                      503 non-null    object 
 2   GICS Sector                   503 non-null    object 
 3   GICS Sub-Industry             503 non-null    object 
 4   Headquarters Location         503 non-null    object 
 5   Date added                    503 non-null    object 
 6   CIK                           503 non-null    int64  
 7   Founded                       503 non-null    object 
 8   sector                        501 non-null    object 
 9   industry                      501 non-null    object 
 10  description                   501 non-null    object 
 11  sharesOutstanding             501 non-null    float64
 12  averageVolume10days           501 non-null    float64
 13  avera